In [42]:
import cdflib
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from pathlib import Path
os.chdir(r'D:/ISRO Hackthon')

In [ ]:
data_folder = Path("HaloCMEs/Data") #Creat4es a Path object pointing to the directory
all_data = []

for file_path in sorted(data_folder.glob("*.cdf")): #Iterates through every file with CDF format
    print(f"Reading {file_path.name}")
    try:
        cdf_file = cdflib.CDF(str(file_path)) 
        try:
            timestamps = cdf_file.varget('epoch_for_cdf_mod') #Newer files
        except:
            timestamps = cdf_file.varget('epoch_for_cdf') #Older files (names were different idk why)
        
        datetime_vals = cdflib.cdfepoch.to_datetime(timestamps) #Coverting the data to numpy64 format
        datetime_python = pd.to_datetime(datetime_vals) #Coverting the numpy64 format to our understanding of date and time YYMMDD
        
        proton_density = cdf_file.varget('proton_density')

        try:
            proton_speed = cdf_file.varget('proton_bulk_speed') #Newer files
        except:
            proton_speed = cdf_file.varget('proton_bulk') #Older files
        
        proton_temp = cdf_file.varget('proton_thermal')

        #Creating a data frame using the pandas library. A Data Frame is a like an excel with rows and columns
        df_day = pd.DataFrame({
            'datetime': datetime_python,
            'proton_density': proton_density,
            'proton_speed': proton_speed,
            'proton_temperature': proton_temp
        })

        df_day.replace(-1.0e+31, pd.NA, inplace = True) #Sometimes CDF files use extremely large negative numbers to represent missing values
        #The part "pd.NA" is way of marking missing data
        df_day.dropna(inplace=True) #Removes any row with atleast one missing value after you make it null using the pandas library

    except Exception as e:
        print(f"Error Reading {file_path.name}")
df = pd.concat(all_data).sort_values('datetime')
df.set_index('datetime', inplace=True)

# Plotting
df.plot(subplots=True, figsize=(12, 8), title="SWIS Parameters Over Time")
plt.tight_layout()
plt.legend()
plt.show()

Reading AL1_ASW91_L2_BLK_20240801_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240802_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240803_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240804_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240805_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240806_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240807_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240808_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240809_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240810_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240811_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240812_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240813_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240814_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240815_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240816_UNP_9999_999999_V01.cdf
Reading AL1_ASW91_L2_BLK_20240817_UNP_9999_999999_V01.cdf
Reading AL1_AS

ValueError: No objects to concatenate

In [ ]:
# Check first few CDF files to list available variables
for file_path in sorted(Path("HaloCMEs/Data").glob("*.cdf")):
    print(f"\n📁 File: {file_path.name}")
    try:
        cdf = cdflib.CDF(str(file_path))
        variables = cdf.cdf_info().zVariables
        print("🔍 Available Variables:")
        for v in variables:
            print("   -", v)
    except Exception as e:
        print(f"❌ Error reading {file_path.name}: {e}")


In [64]:
data_folder = Path("HaloCMEs/SOHOData")
halo_cme_times = []

for file_path in sorted(data_folder.glob("*.txt")):
    with open(file_path, "r") as file:
        for line in file:
            if "Halo" in line and "360" in line:
                try:
                    date_str = line[0:10].strip()
                    time_str = line[11:19].strip()
                    dt = datetime.strptime(f"{date_str} {time_str}", "%Y/%m/%d %H:%M:%S")
                    halo_cme_times.append(dt)
                except Exception as e:
                    print(f"Error parsing line in {file_path.name}:{line.strip()}")
                    print(f"    {e}")

print("Total Halo CME Events found: ", len(halo_cme_times))
print("First 5 events: ")
for dt in halo_cme_times[:5]:
    print(" ->", dt)

Total Halo CME Events found:  113
First 5 events: 
 -> 2024-01-02 11:12:00
 -> 2024-01-04 00:00:00
 -> 2024-01-20 06:24:00
 -> 2024-01-20 09:12:00
 -> 2024-01-21 00:24:00


In [ ]:
# Number of events to visualize (you can change this)
num_events_to_plot = 113

# Make sure df is your SWIS dataframe with datetime index
for i, cme_time in enumerate(halo_cme_times[:num_events_to_plot]):
    start_time = cme_time - timedelta(hours=6)
    end_time = cme_time + timedelta(hours=6)
    
    window_df = df.loc[(df.index >= start_time) & (df.index <= end_time)]

    if window_df.empty:
        print(f"⚠️ No SWIS data for CME at {cme_time}")
        continue

    print(f"🛰️ CME {i+1} → {cme_time} | Records found: {len(window_df)}")

    window_df.plot(
        subplots=True,
        figsize=(12, 8),
        title=f'SWIS Parameters around CME on {cme_time.strftime("%Y-%m-%d %H:%M")}',
        sharex=True
    )
    
    plt.tight_layout()
    plt.show()


In [ ]:
for i, cme_time in enumerate(halo_cme_times[:100]):
    start_time = cme_time - timedelta(hours=6)
    end_time = cme_time + timedelta(hours=6)

    # Relaxed filtering: Find any data within the window
    window_df = df[(df.index >= start_time) & (df.index <= end_time)]

    if len(window_df) == 0:
        continue  # No data for this event

    print(f"🛰️ CME {i+1}: {cme_time} | SWIS records: {len(window_df)}")

    window_df.plot(
        subplots=True,
        figsize=(12, 8),
        title=f'SWIS Parameters around CME on {cme_time.strftime("%Y-%m-%d %H:%M")}',
        sharex=True
    )
    
    plt.tight_layout()
    plt.show()

    if i >= 4:  # Stop after 5 successful plots
        break

In [ ]:
matching_cmes = []

for cme_time in halo_cme_times:
    # ±6 hour window
    start = cme_time - timedelta(hours=6)
    end = cme_time + timedelta(hours=6)

    # Check if any SWIS timestamp falls in this range
    if not df.loc[(df.index >= start) & (df.index <= end)].empty:
        matching_cmes.append(cme_time)

# Results
print(f"✅ CME timestamps that overlap with SWIS data: {len(matching_cmes)}")
for dt in matching_cmes[:5]:
    print("  →", dt)

In [ ]:

# Plot all matching CME windows
for i, cme_time in enumerate(matching_cmes):
    start_time = cme_time - timedelta(hours=6)
    end_time = cme_time + timedelta(hours=6)

    window_df = df[(df.index >= start_time) & (df.index <= end_time)]

    if window_df.empty:
        print(f"⚠️ No data found for CME {i+1} at {cme_time}")
        continue

    print(f"🛰️ CME {i+1}: {cme_time} | SWIS points: {len(window_df)}")

    window_df.plot(
        subplots=True,
        figsize=(12, 8),
        title=f'SWIS Parameters Around CME {i+1} on {cme_time.strftime("%Y-%m-%d %H:%M")}',
        sharex=True
    )
    plt.tight_layout()
    plt.show()


In [ ]:
def detect_cme_candidates(df, speed_thresh=600, temp_spike=20, dens_spike=2, window=12):
    df = df.copy()

    # Rolling means for smoothing
    df['speed_mean'] = df['proton_speed'].rolling(window=window, center=True).mean()
    df['density_mean'] = df['proton_density'].rolling(window=window, center=True).mean()
    df['temp_mean'] = df['proton_temperature'].rolling(window=window, center=True).mean()

    # Spikes
    df['speed_spike'] = df['proton_speed'] > speed_thresh
    df['temp_spike'] = (df['proton_temperature'] - df['temp_mean']) > temp_spike
    df['dens_spike'] = (df['proton_density'] - df['density_mean']) > dens_spike

    # CME-like if at least 2 out of 3 spike flags are True
    df['cme_like'] = df[['speed_spike', 'temp_spike', 'dens_spike']].sum(axis=1) >= 2

    return df


In [ ]:
df_cme = detect_cme_candidates(df)


In [ ]:
# Get timestamps where CME-like signature was detected
detected_cmes = df_cme[df_cme['cme_like']].index

print(f"📍 Potential CME candidates found: {len(detected_cmes)}")
print("🕒 Sample detected times:")
for t in detected_cmes[:5]:
    print("  →", t)

In [ ]:
from datetime import timedelta

for i, t in enumerate(detected_cmes[:3]):
    window = df_cme[(df_cme.index >= t - timedelta(hours=6)) & (df_cme.index <= t + timedelta(hours=6))]

    window[['proton_density', 'proton_speed', 'proton_temperature']].plot(
        subplots=True, figsize=(12, 8),
        title=f'Auto-Detected CME-like Event #{i+1} at {t.strftime("%Y-%m-%d %H:%M")}'
    )
    plt.tight_layout()
    plt.show()